In [3]:
!pip install --quiet transformers
!pip install --quiet tensorboard
!pip install --quiet sentence_transformers
import pandas as pd
import numpy as np
import torch
import pandas as pd
from time import time
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import tensorflow_hub as hub # For USE
from sentence_transformers import SentenceTransformer
import os
import tensorflow as tf
import datetime
from tensorboard.plugins import projector

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Calculating Ans+Ques embeddings and saving the results

In [13]:
t = time()
ques_embed = pd.read_csv("drive/MyDrive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/Questions/output.tsv", sep="\t")
ans_embed = pd.read_csv("drive/MyDrive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/Answers/output.tsv", sep="\t")
print("Time taken: " , int((time() - t)%60) ,"s, ", int((time() - t)/60), "m")

Time taken:  58 s,  0 m


In [14]:
ques_mdata = pd.read_csv("drive/MyDrive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/Questions/metadata.tsv", sep="\t")
ans_mdata = pd.read_csv("drive/MyDrive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/Answers/metadata.tsv", sep="\t")

In [15]:
print(ans_embed.shape)
ans_embed.head(3)

(131123, 768)


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.035330,-0.020545,-0.001838,-0.027146,0.006165,-0.036033,0.030608,-0.000926,0.011802,0.014549,...,-0.040341,0.034209,0.005733,-0.045824,0.008013,0.054945,0.005041,-0.017952,-0.028490,0.024700
1,0.035330,-0.020545,-0.001838,-0.027146,0.006165,-0.036033,0.030608,-0.000926,0.011802,0.014549,...,-0.040341,0.034209,0.005733,-0.045824,0.008013,0.054945,0.005040,-0.017952,-0.028490,0.024700
2,-0.012045,0.000198,-0.011156,0.018694,-0.007562,-0.006029,0.010348,0.049191,0.067288,0.022844,...,-0.057457,-0.015778,-0.038826,-0.018607,0.016415,-0.038477,-0.028977,-0.017798,-0.021441,0.014031


In [16]:
print(ques_embed.shape)
ques_embed.head(3)

(131123, 768)


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.004116,-0.011099,-0.027124,0.014118,-0.016089,-0.010797,-0.015818,-0.012957,0.053849,0.036074,...,-0.062701,0.003592,-0.042352,-0.010929,0.037500,-0.006523,-0.009648,-0.005284,0.016632,-0.007016
1,-0.019652,-0.044659,-0.005455,0.011648,-0.054620,0.014292,-0.022721,-0.038474,0.053255,-0.033202,...,-0.017141,-0.025261,-0.007876,0.010629,-0.003425,-0.087779,-0.004977,-0.039038,0.024891,-0.007417
2,0.009997,-0.076324,-0.041757,0.024525,-0.013982,0.002505,0.051043,-0.007574,0.002133,0.050543,...,-0.000639,-0.020366,0.016907,0.000652,0.016884,-0.004217,-0.051596,-0.022530,-0.004772,-0.027629


In [40]:
embeds = ans_embed + ques_embed
print(embeds.shape)
embeds.head(3)

(131123, 768)


,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.031213,-0.031644,-0.028962,-0.013028,-0.009925,-0.046830,0.014790,-0.013882,0.065651,0.050623,...,-0.103042,0.037801,-0.036619,-0.056753,0.045513,0.048422,-0.004607,-0.023236,-0.011858,0.017684
1,0.015678,-0.065204,-0.007293,-0.015498,-0.048455,-0.021741,0.007887,-0.039400,0.065057,-0.018654,...,-0.057482,0.008948,-0.002144,-0.035195,0.004588,-0.032834,0.000063,-0.056990,-0.003599,0.017283
2,-0.002048,-0.076126,-0.052913,0.043218,-0.021544,-0.003524,0.061390,0.041618,0.069421,0.073387,...,-0.058096,-0.036143,-0.021919,-0.017955,0.033298,-0.042693,-0.080573,-0.040328,-0.026213,-0.013597


In [41]:
print(ques_mdata.shape)
ques_mdata.head(3)

(131123, 1)


,0
0,How many π bonds are present in ferrocene
1,What is covalency of chlorine atom in second e...
2,In solid X atoms goes to corner of the cube an...


In [42]:
print(ans_mdata.shape)
ans_mdata.head(3)

(131123, 1)


,0
0,a
1,a
2,No of x atoms 8 18 2 12 2


In [43]:
mdata = ques_mdata+'? '+ans_mdata
print(mdata.shape)
mdata.sample(3)

(131123, 1)


,0
85186,We cannot touch and see ____________ ? Software
28905,A student tested a compound of chlorine and hy...
87117,May is my favourite month a subject b predicat...


In [44]:
embeds

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.031213,-0.031644,-0.028962,-0.013028,-0.009925,-0.046830,0.014790,-0.013882,0.065651,0.050623,...,-0.103042,0.037801,-0.036619,-0.056753,0.045513,0.048422,-0.004607,-0.023236,-0.011858,0.017684
1,0.015678,-0.065204,-0.007293,-0.015498,-0.048455,-0.021741,0.007887,-0.039400,0.065057,-0.018654,...,-0.057482,0.008948,-0.002144,-0.035195,0.004588,-0.032834,0.000063,-0.056990,-0.003599,0.017283
2,-0.002048,-0.076126,-0.052913,0.043218,-0.021544,-0.003524,0.061390,0.041618,0.069421,0.073387,...,-0.058096,-0.036143,-0.021919,-0.017955,0.033298,-0.042693,-0.080573,-0.040328,-0.026213,-0.013597
3,0.015678,-0.065204,-0.007293,-0.015498,-0.048455,-0.021741,0.007887,-0.039400,0.065057,-0.018654,...,-0.057482,0.008948,-0.002144,-0.035195,0.004588,-0.032834,0.000063,-0.056990,-0.003599,0.017283
4,0.015382,-0.115582,0.000647,-0.016658,-0.037679,0.043268,0.032845,0.022018,0.066186,-0.060700,...,-0.077647,-0.071074,-0.081709,-0.063011,-0.011058,-0.094454,-0.013941,-0.052296,-0.121317,-0.001168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131118,0.042520,-0.250391,-0.040937,0.041368,-0.027111,0.016442,-0.085118,0.036535,-0.046663,0.117321,...,-0.001692,0.001029,-0.032611,0.036223,0.060938,0.048122,0.001430,0.014851,-0.052515,-0.050251
131119,0.042520,-0.250391,-0.040937,0.041368,-0.027111,0.016442,-0.085118,0.036535,-0.046663,0.117321,...,-0.001692,0.001029,-0.032611,0.036223,0.060938,0.048122,0.001430,0.014851,-0.052515,-0.050251
131120,0.088196,0.094445,0.007927,0.001457,-0.004408,-0.023372,-0.100299,-0.061824,-0.026034,-0.060561,...,0.065804,0.037151,-0.086320,-0.003085,0.006946,0.049949,-0.086145,0.065642,0.085490,0.008391
131121,0.126575,0.081320,0.004011,0.065095,-0.038288,-0.032410,-0.091307,-0.046050,-0.018161,-0.007695,...,0.054510,0.063045,-0.070886,0.047733,0.007908,0.028912,-0.082278,0.056710,0.022219,-0.016767


In [45]:
# embeds.to_csv("drive/My Drive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/QnA-Combined/output.tsv")
# mdata.to_csv("drive/My Drive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/QnA-Combined/metadata.tsv")

In [46]:
embeds = pd.DataFrame(embeds)
path = "drive/My Drive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/QnA-Combined/output.tsv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  embeds.to_csv(f, index=False, sep='\t')

In [47]:
mdata = pd.DataFrame(mdata)
path = "drive/My Drive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/QnA-Combined/metadata.tsv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  mdata.to_csv(f, index=False, sep='\t')

#### Analyzing the new embeddings

In [10]:
embeds = pd.read_csv("drive/My Drive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/QnA-Combined/output.tsv")
mdata = pd.read_csv("drive/My Drive/Extramarks Project/Dataset versions/embeddings_EM_answers_and_questions/QnA-Combined/metadata.tsv")

In [37]:
embeds

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.031213,-0.031644,-0.028962,-0.013028,-0.009925,-0.046830,0.014790,-0.013882,0.065651,0.050623,...,-0.103042,0.037801,-0.036619,-0.056753,0.045513,0.048422,-0.004607,-0.023236,-0.011858,0.017684
1,0.015678,-0.065204,-0.007293,-0.015498,-0.048455,-0.021741,0.007887,-0.039400,0.065057,-0.018654,...,-0.057482,0.008948,-0.002144,-0.035195,0.004588,-0.032834,0.000063,-0.056990,-0.003599,0.017283
2,-0.002048,-0.076126,-0.052913,0.043218,-0.021544,-0.003524,0.061390,0.041618,0.069421,0.073387,...,-0.058096,-0.036143,-0.021919,-0.017955,0.033298,-0.042693,-0.080573,-0.040328,-0.026213,-0.013597
3,0.015678,-0.065204,-0.007293,-0.015498,-0.048455,-0.021741,0.007887,-0.039400,0.065057,-0.018654,...,-0.057482,0.008948,-0.002144,-0.035195,0.004588,-0.032834,0.000063,-0.056990,-0.003599,0.017283
4,0.015382,-0.115582,0.000647,-0.016658,-0.037679,0.043268,0.032845,0.022018,0.066186,-0.060700,...,-0.077647,-0.071074,-0.081709,-0.063011,-0.011058,-0.094454,-0.013941,-0.052296,-0.121317,-0.001168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.027419,-0.073134,0.019010,0.016804,0.003582,0.022742,-0.035019,0.043495,0.093361,-0.087957,...,-0.045062,-0.039524,0.048983,0.039536,-0.017223,-0.023811,-0.019727,0.004740,-0.069868,-0.036547
996,0.005030,-0.117530,-0.001906,0.002218,-0.068175,-0.020818,0.061739,0.060526,0.038695,-0.045317,...,-0.062994,-0.049257,0.108795,0.055104,-0.043102,-0.066869,-0.017999,-0.024888,-0.104583,-0.021896
997,-0.037679,-0.111173,0.002297,-0.042308,-0.112457,0.069223,-0.102168,0.121811,0.168561,-0.043764,...,-0.046639,-0.076039,0.044702,0.031491,-0.030702,0.000066,-0.097446,0.000121,-0.094153,-0.056435
998,-0.021459,-0.080990,-0.036351,-0.016725,-0.064626,-0.063744,-0.112044,0.004728,0.028095,0.023796,...,-0.106342,0.013735,-0.022197,0.020044,0.002684,0.033301,0.040631,-0.006199,0.020411,-0.000410
